In [32]:
from IPython.core.display import HTML
HTML("""
<style>

div.cell { /* Tunes the space between cells */
margin-top:1em;
margin-bottom:1em;
}

div.text_cell_render h1 { /* Main titles bigger, centered */
font-size: 1.8em;
line-height:1.0em;
text-align:center;
}

div.text_cell_render h2 { /*  Parts names nearer from text */
margin-bottom: -0.4em;
}


div.text_cell_render { /* Customize text cells */
font-family: 'Times New Roman';
font-size:1.4em;
line-height:1.3em;
padding-left:2em;
padding-right:2em;
}
</style>
""")

# Классификация рукописных цифр с помощью TensorFlow

![](images/mnist.jpeg)

Классическая задача распознавания рукописных цифр на основе набора данных [MNIST](http://yann.lecun.com/exdb/mnist/)

## Постановка задачи

- 60_000 изображений с цифрами
- Отцентрированы
- Черно-белые
- Размер 28x28 пикселя

# Простейшая однослойная нейросеть

![](images/one_layer.png)

Изображение цифры, размером 28x28 уплощается в вектор размера 784 пикселей, который подается на вход однослойной нейросети. Каждый нейрон по отдельности проивзодит взвешенную сумму всех входных сигналов, добавляет некоторый bias-порог и пропускает результат через некоторую функцию активации.

![](images/perceptron_schematic.png)

### Функция перцептрона

$$Y = g(X.W + b) $$

$$sigmoid(z_n) = \frac{1}{1 + e^{z_n}} $$

$$softmax(z_n) = \frac{e^{z_n}}{||{e^z}||} $$

![](images/activation_functions.png)

Так как функции активации по определению возвращают числа от 0 до 1, то мы получаем, что фактически наш перцептрон возвращает вероятности принадлежности к классу.

![](images/eval_single_layer.png)

![](images/softmax-regression-vectorequation.png)

### Тренировка сети

$$H_{y'}(y) = - \sum{y'_i log(y_i)}$$

y - оригинальный  вектор

y' - предсказанный  вектор

![](images/cross_entropy.png)

## Практика 0

- импортировать TF
- переменные, константы, подстановки в TF
- запуск вычислений

In [1]:
import tensorflow as tf
tf.set_random_seed(0)

Создание констант

In [6]:
node1 = tf.constant(3.0)
node2 = tf.constant(4.0)
print(node1, node2)

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32)


Запуск вычислений

In [55]:
sess = tf.InteractiveSession()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [10]:
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add_1:0", shape=(), dtype=float32)
sess.run(node3): 7.0


Создание подстановок

In [11]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b # переопределенный оператор для tf.add

In [13]:
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


Создание переменных

In [16]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b
print(linear_model)

Tensor("add_3:0", dtype=float32)


In [18]:
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [22]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_deltas) # функция потерь - или ошибки
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


In [23]:
# Поправим вручную потери линейного алгоритма.
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


## Практика 1

- импортировать TF
- загрузить dataset
- описать структуру сети
- запустить кросс валидацию

### Формулы для реализации

$$Y = softmax(X.W + b) $$

$$softmax(z_n) = \frac{e^{z_n}}{||{e^z}||} $$

$$H_{y'}(y) = - \sum{y'_i log(y_i)}$$

In [20]:
import tensorflow as tf
tf.set_random_seed(0)
tf.reset_default_graph()

In [21]:
with tf.name_scope('X'):
    X = tf.placeholder(tf.float32, [None, 784], name='X')
    x_image = tf.reshape(X, [-1, 28, 28, 1])
with tf.name_scope('weights'):
    W = tf.Variable(tf.zeros([784, 10]), name='weights')
with tf.name_scope('biases'):
    b = tf.Variable(tf.zeros([10]), name='biases')
    
tf.summary.image('input', x_image, 3)
tf.summary.histogram("weights", W)
tf.summary.histogram("biases", b)

<tf.Tensor 'biases_1:0' shape=() dtype=string>

In [22]:
with tf.name_scope('Wx_plus_b'):
    # Модель Y = X.W + b
    Y = tf.nn.softmax(tf.matmul(tf.reshape(X, [-1, 784]), W) + b, name='labels')

# Подстановка для корректных значений входных данных
Y_ = tf.placeholder(tf.float32, [None, 10], name='Y_')

with tf.name_scope('total'):
    # Функция потерь H = Sum(Y_ * log(Y))
    cross_entropy = -tf.reduce_sum(Y_ * tf.log(Y), name='xentropy')

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        # Доля верных ответов найденных в наборе
        is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32), name='xentropy_mean')

tf.summary.scalar('accuracy', accuracy)
tf.summary.scalar('cross_entropy', cross_entropy)

<tf.Tensor 'cross_entropy:0' shape=() dtype=string>

In [23]:
with tf.name_scope('train'):
    # Оптимизируем функцию потерь меотодом градиентного спуска
    # 0.003 - это шаг градиента, гиперпараметр
    optimizer = tf.train.GradientDescentOptimizer(0.003)
    # Минимизируем потери
    train_step = optimizer.minimize(cross_entropy)

In [5]:
# Загружаем тренировочный датасет MNIST
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True, reshape=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [25]:
def save_summary(sess, mnist, writer, merged, accuracy, i):
    if i % 10 == 0:
        test_data={X: mnist.test.images, Y_: mnist.test.labels}
        summary, a = sess.run([merged, accuracy], feed_dict=test_data)
        writer.add_summary(summary, i)
        if i % 200 == 0:
            print("Test: {}".format(a))

In [27]:
with tf.Session() as sess:
    merged = tf.summary.merge_all() # Merge all the summaries and write them out to
    writer = tf.summary.FileWriter("/tmp/tensorflow/one_layer.log", sess.graph)
    tf.global_variables_initializer().run()

    for i in range(1000):
        # загружаем набор изображений и меток классов
        batch_X, batch_Y = mnist.train.next_batch(100)
        train_data={X: batch_X, Y_: batch_Y}

        # train
        sess.run(train_step, feed_dict=train_data)
        
        save_summary(sess, mnist, writer, merged, accuracy, i)
        
    writer.close()

Test: 0.43070000410079956
Test: 0.9021000266075134
Test: 0.9103000164031982
Test: 0.9164999723434448
Test: 0.914900004863739


In [74]:
!ls /tmp/tensorflow/

one_layer.log


In [30]:
def single_layer(mnist):
    X = tf.placeholder(tf.float32, [None, 784], name='X')
    W = tf.Variable(tf.zeros([784, 10]), name='weights')
    b = tf.Variable(tf.zeros([10]), name='biases')

    Y = tf.nn.softmax(tf.matmul(tf.reshape(X, [-1, 784]), W) + b, name='labels')
    Y_ = tf.placeholder(tf.float32, [None, 10], name='Y_')

    cross_entropy = -tf.reduce_sum(Y_ * tf.log(Y), name='xentropy')
    is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32), name='xentropy_mean')

    optimizer = tf.train.GradientDescentOptimizer(0.003)
    train_step = optimizer.minimize(cross_entropy)

    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(1000):
            batch_X, batch_Y = mnist.train.next_batch(100)
            train_data={X: batch_X, Y_: batch_Y}

            # train
            sess.run(train_step, feed_dict=train_data)

            if i % 100 == 0:
                test_data={X: mnist.test.images, Y_: mnist.test.labels}
                a, c = sess.run([accuracy, cross_entropy], feed_dict=test_data)
                print("Test: {}, {}".format(a, c))

In [31]:
single_layer(mnist)

Test: 0.27900001406669617, 20985.52734375
Test: 0.8912000060081482, 4185.6044921875
Test: 0.8966000080108643, 3815.54150390625
Test: 0.9068999886512756, 3428.35595703125
Test: 0.9078999757766724, 3309.99658203125
Test: 0.9114999771118164, 3185.62109375
Test: 0.9142000079154968, 3103.473388671875
Test: 0.9156000018119812, 3059.9453125
Test: 0.916100025177002, 3024.944091796875
Test: 0.9162999987602234, 2978.724853515625


## Практика 2

- Многослойная нейросеть